In [15]:
import keras
import json
import numpy as np
from keras_preprocessing.text import tokenizer_from_json
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

{'<OOV>': 1, 'otc': 2, 'recommend': 3, 'biogesic': 4, 'take': 5, 'medicine': 6, 'available': 7, 'pain': 8, 'over': 9, 'the': 10, 'counter': 11, 'relief': 12, 'need': 13, 'talk': 14, 'medication': 15, 'understand': 16, 'want': 17, 'would': 18, 'something': 19, 'human': 20, 'suggest': 21, 'option': 22, 'looking': 23, 'child': 24, 'safe': 25, 'helping': 26, 'reliever': 27, 'contact': 28, 'person': 29, 'occasional': 30, 'recommendations': 31, 'agent': 32, 'aid': 33, 'symptoms': 34, 'cough': 35, 'remedy': 36, 'fever': 37, 'today': 38, 'like': 39, 'allergy': 40, 'experiencing': 41, 'options': 42, 'cold': 43, 'children': 44, 'trouble': 45, 'prepared': 46, 'headache': 47, 'going': 48, 'someone': 49, 'currently': 50, 'prescription': 51, 'purchase': 52, 'joint': 53, 'certain': 54, 'seasonal': 55, 'allergies': 56, 'bloating': 57, 'sleep': 58, 'suggestions': 59, 'solution': 60, 'trip': 61, 'medications': 62, 'taking': 63, 'feeling': 64, 'stressed': 65, 'stress': 66, 'minor': 67, 'get': 68, 'dosage

In [32]:
#Load models ang tokenizer
from keras.models import Model, Sequential
from keras.layers import Softmax

intent_recognition_model = keras.models.load_model('intent_recognition_model.keras')
probability_model = Sequential([intent_recognition_model, Softmax()])

#Parameters of Tokenizer
vocab_size = 1000
max_length = 50
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

#Load tokenizer
with open('tokenizer.json') as f:
    data = json.load(f)
    tokenizer = tokenizer_from_json(data)

print(tokenizer.word_index)

{'<OOV>': 1, 'otc': 2, 'recommend': 3, 'biogesic': 4, 'take': 5, 'medicine': 6, 'available': 7, 'pain': 8, 'over': 9, 'the': 10, 'counter': 11, 'relief': 12, 'need': 13, 'talk': 14, 'medication': 15, 'understand': 16, 'want': 17, 'would': 18, 'something': 19, 'human': 20, 'suggest': 21, 'option': 22, 'looking': 23, 'child': 24, 'safe': 25, 'helping': 26, 'reliever': 27, 'contact': 28, 'person': 29, 'occasional': 30, 'recommendations': 31, 'agent': 32, 'aid': 33, 'symptoms': 34, 'cough': 35, 'remedy': 36, 'fever': 37, 'today': 38, 'like': 39, 'allergy': 40, 'experiencing': 41, 'options': 42, 'cold': 43, 'children': 44, 'trouble': 45, 'prepared': 46, 'headache': 47, 'going': 48, 'someone': 49, 'currently': 50, 'prescription': 51, 'purchase': 52, 'joint': 53, 'certain': 54, 'seasonal': 55, 'allergies': 56, 'bloating': 57, 'sleep': 58, 'suggestions': 59, 'solution': 60, 'trip': 61, 'medications': 62, 'taking': 63, 'feeling': 64, 'stressed': 65, 'stress': 66, 'minor': 67, 'get': 68, 'dosage

In [26]:
#Remove the stopwords
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import contractions

stop_words = set(stopwords.words('english'))

def removeStopWords(sentence):
    filtered_result = ""
    sent_tokenized = contractions.fix(sentence)
    sent_tokenized = word_tokenize(sent_tokenized)
    #print(f"sent_tokenized: {sent_tokenized}")
    for word in sent_tokenized:
        #print(f"Checking if {word} is a stop_word")
        if word.casefold() not in stop_words:
            filtered_result = filtered_result + " "+ word
    filtered_result = filtered_result.strip()
    #print(filtered_result)
    return filtered_result

In [39]:
def getIntent(user_prompt):
    print(f"User Prompt: {user_prompt}")
    clean_input = removeStopWords(user_prompt)
    print(f"USer Prompt Without Stopwords: {clean_input}")
    tokens = tokenizer.texts_to_sequences([clean_input])
    print(f"Tokens: {tokens}")
    tokens = pad_sequences(tokens, maxlen=max_length, padding=padding_type)
    probabilities = probability_model.predict(tokens)
    print(f"Probabilities: {probabilities}")
    predicted_label = np.argmax(probabilities)
    return predicted_label

#0: Contact Human Agent - Ask for live human agent
#1: Medicine Suggestion Query - Ask for suggestion of OTC medicines based from symptoms or generic drug name (ex. do you have anything for flu?; do you have vitamin D supplements?). 
#2: Medicine Dosage Query - Ask for dosage directions
#3: Medicine Availability Query - 

def GenerateBotResponse(intent):
    if (intent == 0):
        return "We will get a human agent to talk with you as soon as possible."
    elif (intent == 1):
        return "Here is the medicine you asked for."
    elif (intent == 2):
        return "Here are the instructions for taking your medicine:"
    elif (intent == 3):
        return "Here is the availability of the medicine you asked for."
    else:
        return "I'm sorry, I don't understand."

In [41]:
#Test
test_prompt = "Do you have betadine?"

print(GenerateBotResponse(getIntent(test_prompt)))

User Prompt: Do you have betadine?
USer Prompt Without Stopwords: betadine ?
Tokens: [[209]]
1/1 [==============================] - 0s 25ms/step
Probabilities: [[0.33217418 0.02884594 0.29395673 0.3427316  0.00229156]]
Here is the availability of the medicine you asked for.


In [ ]:
GenerateBotResponse(getIntent(user_input))